In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using JLD2
include("ETO.jl")
include("RO.jl")
include("Data.jl")
include("Estimate.jl")
include("Performance.jl")

greedy_search_optimal_price (generic function with 1 method)

In [2]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(grand_pa_dir, "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end

## Functions

In [3]:
function generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar)
    Input_Data = Dict()
    for iter in 1:iterations
        A_true, B_true = Generate_Coef(N_u, N,max_offdiag,offdiag_sign);
        U_train, P_train = Generate_Feat_Data(N_u, N, S_train);
        U_test, P_test = Generate_Feat_Data(N_u, N, S_test);

        Input_Data["iter=$(iter)_Obs_Feat"] = U_test[1,:];
        Input_Data["iter=$(iter)_A_true"] = A_true;
        Input_Data["iter=$(iter)_B_true"] = B_true;
        Input_Data["iter=$(iter)_P_dag"] = round.(rand(N, K) .* P_bar; digits=2);
        Input_Data["iter=$(iter)_U_train"] = U_train;
        Input_Data["iter=$(iter)_P_train"] = P_train;

        A_hat, B_hat = Estimate_MNL_Para(U_train, P_train, S_train, N, N_u, N, A_true, B_true);

        Input_Data["iter=$(iter)_A_hat"] = A_hat
        Input_Data["iter=$(iter)_B_hat"] = B_hat
    end
    return Input_Data
end

generate_Input_Data (generic function with 1 method)

In [4]:
function Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag)
    RST_ = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        if print_flag
            println("iter=$(iter): rev_Oracle = ",round(rev_ETO,digits=6),", price_Oracle = ",price_ETO)
        end
        RST_["iter=$(iter)_Time"] = time_ETO
        RST_["iter=$(iter)_Rev"] = rev_ETO
        RST_["iter=$(iter)_Price"] = price_ETO
        RST_["iter=$(iter)_Obj"] = obj_ETO
    end
    return RST_
end

Run_Oracle (generic function with 1 method)

In [5]:
function Run_ETO(iterations, N, N_u, K, Input_Data,print_flag)
    RST_ETO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_hat,B_hat,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        if print_flag
            println("iter=$(iter): rev_ETO = ",round(rev_ETO,digits=6),", price_ETO = ",price_ETO)
        end
        RST_ETO["iter=$(iter)_Time"] = time_ETO
        RST_ETO["iter=$(iter)_Rev"] = rev_ETO
        RST_ETO["iter=$(iter)_Price"] = price_ETO
        RST_ETO["iter=$(iter)_Obj"] = obj_ETO
    end
    return RST_ETO
end

Run_ETO (generic function with 1 method)

In [6]:
function search_opt_price(N,p_lb,p_ub,b_n)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, price[1:N])                      # y_{nk}
    @constraint(model, price .>= p_lb)
    @constraint(model, price .<= p_ub)
    @objective(model, Max,b_n' * price )
    optimize!(model)
    obj_val = objective_value(model)
    return obj_val
end

search_opt_price (generic function with 1 method)

In [7]:
function Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data)
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        # A_true = Input_Data["iter=$(iter)_A_true"]
        # B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]

        for RO_coef in RO_coef_all
            A_lb = A_hat .- RO_coef .* abs.(A_hat);
            A_ub = A_hat .+ RO_coef .* abs.(A_hat);
            B_lb = B_hat .- RO_coef .* abs.(B_hat);
            B_ub = B_hat .+ RO_coef .* abs.(B_hat);
            
            p_ub = vec(maximum(P_dag,dims=2))
            p_lb = vec(minimum(P_dag,dims=2))
            p_max = maximum(p_ub)
            p_min = minimum(p_lb)
            Obs_Feat_Trun = [max(-Obs_Feat[ind],0) for ind in 1:N_u]
            psi_lb = zeros(N)
            for n in 1:N
                b_n = B_lb[n,:]
                obj_n = search_opt_price(N,p_lb,p_ub,b_n)
                psi_lb[n] = -exp(-Obs_Feat_Trun' * (A_ub[n,:] - A_lb[n,:]) + Obs_Feat' * A_lb[n,:] + obj_n)*(p_max-p_min)
            end        

            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_lb"] = psi_lb
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_ub"] = zeros(N)
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_lb"] = [p_min - p_max for i in 1:N]
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_ub"] = zeros(N)

            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_lb"] = A_lb
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_ub"] = A_ub
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_lb"] = B_lb
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_ub"] = B_ub
        end
    end
    return Input_Data
end

Calculate_Hyper_Param (generic function with 1 method)

In [8]:
function Run_RO(RO_coef, iterations, N, N_u, K, Input_Data,psi_coef,model_name,print_flag)
    RST_RO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]

        psi_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_lb"]
        psi_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_ub"]
        phi_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_lb"]
        phi_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_ub"]

        A_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_lb"]
        A_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_ub"]
        B_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_lb"]
        B_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_ub"]
        if model_name == "Two_Side"
            obj_RO,X_RO,time_RO = Solve_RO(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub,phi_lb,phi_ub)
        else
            obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)
        end
        
        rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, Obs_Feat, P_dag)
        if print_flag
            println("iter=$(iter): rev_RO = ",round(rev_RO,digits=6),", price_RO = ",price_RO)
        end
        RST_RO["iter=$(iter)_Time"] = time_RO
        RST_RO["iter=$(iter)_Rev"] = rev_RO
        RST_RO["iter=$(iter)_Price"] = price_RO
        RST_RO["iter=$(iter)_Obj"] = obj_RO
    end
    return RST_RO
end

Run_RO (generic function with 1 method)

In [9]:
function Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, psi_coef,model_name,print_flag)
    RST_RO = Run_RO(RO_coef, iterations, N, N_u, K, Input_Data,psi_coef,model_name,print_flag);
    return RST_RO
end

Run_RO_with_RO_coef (generic function with 1 method)

In [10]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    Params["offdiag_sign"] = offdiag_sign
    Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    Params["RO_coef_all"] = RO_coef_all
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

## Evaluate

#### Baseline Parameters

In [11]:
project_name = "Assortment_Pricing_Simulated_Data/"
seed = 1;
N = 5;
N_u = 1;
K = 10;
S_train = 100;
S_test = 1000;
offdiag_sign = "mix";
max_offdiag = 0.3;
P_bar = 1.0;
iterations = 10;
RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.14,0.16,0.18,0.2];

### offdiag is mix

In [12]:
offdiag_sign = "mix";
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

Dict{Any, Any} with 10 entries:
  "RO_coef_all"  => [0.0, 0.01, 0.05, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2]
  "S_train"      => 100
  "N"            => 5
  "S_test"       => 1000
  "seed"         => 1
  "P_bar"        => 1.0
  "N_u"          => 1
  "K"            => 10
  "offdiag_sign" => "mix"
  "max_offdiag"  => 0.3

In [13]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_name,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);

- Generate Data

In [14]:
Random.seed!(seed)
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
save(string(this_data_file, "Input_Data.jld2"), Input_Data);

- Run Oracle

In [15]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

iter=1: rev_Oracle = 0.732646, price_Oracle = [0.76, 0.86, 0.96, 0.99, 0.81]
iter=2: rev_Oracle = 0.749816, price_Oracle = [0.97, 0.95, 0.98, 1.0, 0.97]
iter=3: rev_Oracle = 0.668777, price_Oracle = [0.89, 0.97, 0.99, 0.81, 0.85]
iter=4: rev_Oracle = 0.691708, price_Oracle = [0.98, 0.88, 0.99, 0.64, 0.84]
iter=5: rev_Oracle = 0.681864, price_Oracle = [0.95, 0.78, 0.93, 0.84, 0.92]
iter=6: rev_Oracle = 0.696875, price_Oracle = [1.0, 0.89, 0.88, 0.91, 0.91]
iter=7: rev_Oracle = 0.611267, price_Oracle = [0.9, 0.81, 0.63, 0.99, 0.82]
iter=8: rev_Oracle = 0.769298, price_Oracle = [0.81, 0.8, 0.98, 0.97, 0.91]
iter=9: rev_Oracle = 0.682768, price_Oracle = [0.82, 0.85, 0.97, 0.97, 0.95]
iter=10: rev_Oracle = 0.699545, price_Oracle = [0.89, 0.89, 0.79, 0.97, 0.97]


- Run ETO

In [16]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

iter=1: rev_ETO = 0.642982, price_ETO = [0.24, 0.86, 0.96, 0.99, 0.81]
iter=2: rev_ETO = 0.643008, price_ETO = [0.97, 0.95, 0.98, 1.0, 0.27]
iter=3: rev_ETO = 0.286645, price_ETO = [0.12, 0.97, 0.99, 0.1, 0.85]
iter=4: rev_ETO = 0.683653, price_ETO = [0.98, 0.88, 0.94, 0.64, 0.84]
iter=5: rev_ETO = 0.559875, price_ETO = [0.95, 0.78, 0.93, 0.29, 0.92]
iter=6: rev_ETO = 0.564284, price_ETO = [1.0, 0.89, 0.88, 0.26, 0.79]
iter=7: rev_ETO = 0.611267, price_ETO = [0.9, 0.81, 0.63, 0.99, 0.82]
iter=8: rev_ETO = 0.551575, price_ETO = [0.07, 0.8, 0.98, 0.97, 0.66]
iter=9: rev_ETO = 0.490063, price_ETO = [0.82, 0.71, 0.97, 0.97, 0.03]
iter=10: rev_ETO = 0.57984, price_ETO = [0.65, 0.89, 0.79, 0.08, 0.97]
Rev_ETO/Rev_Oracle = [0.8776, 0.8576, 0.4286, 0.9884, 0.8211, 0.8097, 1.0, 0.717, 0.7178, 0.8289]


- Run RO under different uncertainty set

In [ ]:
model_name="Two_Side"
RST_RO = Dict(); Rev_RO = Dict();
for RO_coef in RO_coef_all
    println("Running RO with RO_coef = ",RO_coef)
    RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
    RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
end
save(string(this_data_file, "RST_RO.jld2"), RST_RO);

Running RO with RO_coef = 0.0
Running RO with RO_coef = 0.01
Running RO with RO_coef = 0.05
Running RO with RO_coef = 

In [ ]:
for RO_coef in RO_coef_all
    Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
    println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
end

### offdiag is positive

In [ ]:
offdiag_sign = "positive";
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_name,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
Random.seed!(seed)
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
save(string(this_data_file, "Input_Data.jld2"), Input_Data);

In [ ]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
model_name="Two_Side"
RST_RO = Dict(); Rev_RO = Dict();
for RO_coef in RO_coef_all
    println("Running RO with RO_coef = ",RO_coef)
    RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
    RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
end
save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
# for RO_coef in RO_coef_all
#     Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
#     println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
# end

### offdiag is negative

In [ ]:
offdiag_sign = "negative";
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_name,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
model_name="Two_Side"
RST_RO = Dict(); Rev_RO = Dict();
for RO_coef in RO_coef_all
    println("Running RO with RO_coef = ",RO_coef)
    RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
    RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
end
save(string(this_data_file, "RST_RO.jld2"), RST_RO);